In [1]:
import os 

In [2]:
%pwd

'/Users/arihantsingla/Documents/CDD_Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/arihantsingla/Documents/CDD_Project'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path: Path
    training_data: Path
    all_params: dict
    params_batch_size: int
    params_image_size: list


In [6]:
from src.chicken_disease_classification.utils.common import read_yaml, create_directories,save_json
from src.chicken_disease_classification.constants import *

In [7]:
class configManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

    def get_model_evaluation_config(self)->ModelEvaluationConfig:

        eval_config = ModelEvaluationConfig(
            model_path=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/Chicken-fecal-images"),
            all_params=self.params,
            params_batch_size=self.params.training.BATCH_SIZE,
            params_image_size=self.params.prepare_base_model.IMAGE_SIZE)
        
        return eval_config

In [8]:
from urllib.parse import urlparse
import tensorflow as tf

In [9]:
class Evaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config


    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluate(self):
        model = self.load_model(self.config.model_path)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path="scores.json", data=scores)
        

In [10]:
try:
    config = configManager()
    model_evaluation_config = config.get_model_evaluation_config()
    evaluation = Evaluation(config=model_evaluation_config)
    evaluation.evaluate()
    evaluation.save_score()
except Exception as e:
    raise e

[2025-03-22 23:37:29,312: INFO: common: Reading YAML file from: config/config.yaml]
[2025-03-22 23:37:29,314: INFO: common: YAML file read successfully.]
[2025-03-22 23:37:29,314: INFO: common: Reading YAML file from: params.yaml]
[2025-03-22 23:37:29,315: INFO: common: YAML file read successfully.]
[2025-03-22 23:37:29,421: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 116 images belonging to 2 classes.


/Users/arihantsingla/Documents/CDD_Project/CDD/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.2496 - loss: 161.9934
